In [7]:
pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 9.5 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 13.5 MB/s  0:00:00

   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   -------------------- ------------------- 1/2 [sqlalchemy]
   ---------------------------------------- 2/2 [sqlalchemy]

Note: you may need to restart the kernel to use updated packages.


In [8]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

def test_postgres_connection():
    try:
        # Подключаемся к PostgreSQL
        connection = psycopg2.connect(
            host='host.docker.internal',  # или 'host.docker.internal'
            port=5432,
            database='postgres',
            user='airflow',
            password='airflow'
        )
        print("✅ Успешное подключение к PostgreSQL!")

        # Создаем курсор
        cursor = connection.cursor()

        # Выполняем простой запрос
        cursor.execute("SELECT version();")
        version = cursor.fetchone()
        print(f"📊 Версия PostgreSQL: {version[0]}")

        # Получаем список таблиц
        cursor.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
            ORDER BY table_name;
        """)

        tables = cursor.fetchall()
        print("📋 Таблицы в базе данных:")
        for table in tables:
            print(f"   - {table[0]}")

        # Если есть таблицы, читаем первую в DataFrame
        if tables:
            first_table = tables[0][0]
            print(f"\n🎯 Читаем данные из таблицы: {first_table}")

            # Читаем данные в DataFrame
            df = pd.read_sql(f"SELECT * FROM {first_table} LIMIT 5", connection)

            if not df.empty:
                print("✅ Данные успешно загружены в DataFrame!")
                print(f"📐 Размер DataFrame: {df.shape}")
                print("\n📊 Первые строки:")
                display(df.head())
            else:
                print("ℹ️ Таблица пустая")
        else:
            print("ℹ️ В базе нет таблиц в схеме 'public'")

        # Закрываем соединение
        cursor.close()
        connection.close()
        print("\n🔒 Соединение закрыто")

    except Exception as e:
        print(f"❌ Ошибка: {e}")

# Запускаем проверку
test_postgres_connection()

✅ Успешное подключение к PostgreSQL!
📊 Версия PostgreSQL: PostgreSQL 13.22 (Debian 13.22-1.pgdg13+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 14.2.0-19) 14.2.0, 64-bit
📋 Таблицы в базе данных:
   - какой_кайф

🎯 Читаем данные из таблицы: какой_кайф
✅ Данные успешно загружены в DataFrame!
📐 Размер DataFrame: (5, 4)

📊 Первые строки:


C:\Users\mustd\AppData\Local\Temp\ipykernel_28008\1074346173.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f"SELECT * FROM {first_table} LIMIT 5", connection)


,id,кайф,уровень_кайфа,описание_кафа
0,1,компьютер,5,компик играть ммм
1,2,пиво,5,Паулайнер пить
2,3,компьютер,5,компик играть ммм
3,4,пиво,5,Паулайнер пить
4,5,компьютер,5,компик играть ммм



🔒 Соединение закрыто
